# Esercizio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. Analisi del Dataset

predire quante visualizzazione puo fare un video che carico su youtube in base a quello che ha imparato nel dataser

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import sys
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#Import del Dataset e visualizzazione dei dati iniziali

In [4]:
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive')

df = pd.read_csv('/content/drive/MyDrive/dataset/videos-stats.csv', encoding='latin-1')

print(f"Dimensioni del dataset: {df.shape}")
print(f"Totale righe: {df.shape[0]}")
print(f"Totale colonne: {df.shape[1]}")

print(df.head())
#Statisctiche descrittive
print("\n")
print("Statistiche descrittive:")
print(df.describe())

df = pd.read_csv('/content/drive/MyDrive/dataset/videos-stats.csv', encoding='latin-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dimensioni del dataset: (1881, 8)
Totale righe: 1881
Totale colonne: 8
   Unnamed: 0                                              Title     Video ID  \
0           0  Apple Pay Is Killing the Physical Wallet After...  wAZZ-UWGVHI   
1           1                    The most EXPENSIVE thing I own.  b3x28s61q3c   
2           2                 My New House Gaming Setup is SICK!  4mgePWWCAmA   
3           3  Petrol Vs Liquid Nitrogen | Freezing Experimen...  kXiYSI7H2b0   
4           4                     Best Back to School Tech 2022!  ErMwWXQxHp0   

  Published At Keyword    Likes  Comments      Views  
0   2022-08-23    tech   3407.0     672.0   135612.0  
1   2022-08-24    tech  76779.0    4306.0  1758063.0  
2   2022-08-23    tech  63825.0    3338.0  1564007.0  
3   2022-08-23    tech  71566.0    1426.0   922918.0  
4   2022-08-08    tech  96513.0    515

#Enconding

In [5]:
label_encoder = LabelEncoder()

#Applica Label Encoding alla colonna "Title e Keyword"
df['Title'] = label_encoder.fit_transform(df['Title'])
df['Keyword'] = label_encoder.fit_transform(df['Keyword'])

#Normalizzazione delle colonne

In [6]:
#Selezionate le colonne da normalizzare
columns_to_normalize = ['Likes','Comments','Views','Title','Keyword']

#Creare uno scaler e normalizzare i dati
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

#Visualizzazione dei dati dopo l'encoding

In [7]:
#Visualizzare i primi risultati
print(df.head())
print(df.info())

   Unnamed: 0     Title     Video ID Published At  Keyword     Likes  \
0           0  0.138694  wAZZ-UWGVHI   2022-08-23    0.925  0.000207   
1           1  0.826228  b3x28s61q3c   2022-08-24    0.925  0.004669   
2           2  0.598489  4mgePWWCAmA   2022-08-23    0.925  0.003881   
3           3  0.652995  kXiYSI7H2b0   2022-08-23    0.925  0.004352   
4           4  0.170534  ErMwWXQxHp0   2022-08-08    0.925  0.005869   

   Comments     Views  
0  0.000918  0.000034  
1  0.005877  0.000436  
2  0.004556  0.000388  
3  0.001947  0.000229  
4  0.007036  0.000460  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1881 non-null   int64  
 1   Title         1881 non-null   float64
 2   Video ID      1881 non-null   object 
 3   Published At  1881 non-null   object 
 4   Keyword       1881 non-null   float64
 5   Likes      

#Preprocessing e Normalizzazione

In [8]:
#Imputare valori mancanti
#Numerici: sostituire con la media per "Likes", "Comments" e "Views"
for col in ["Likes", "Comments", "Views"]:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

#Rimuovere duplicati
df = df.drop_duplicates()

#Gestione degli outlier
#Identificazione di outlier con il metodo iqr
numeric_cols = df.select_dtypes(include=np.number).columns
if not df.empty:
    q1 = df[numeric_cols].quantile(0.25)
    q3 = df[numeric_cols].quantile(0.75)
    iqr = q3 - q1

    #Rimuovere outlier solo dalle colonne numeriche
    outliers = ((df[numeric_cols] < (q1 - 1.5 * iqr)) | (df[numeric_cols] > (q3 + 1.5 * iqr))).any(axis=1)
    df = df[~outliers]

#Controllo valori mancanti
missing_values = df.isnull().sum()
print("Valori mancanti per colonna:")
print(missing_values)

#Rimozione delle righe con valori mancanti
df.dropna(inplace=True)

Valori mancanti per colonna:
Unnamed: 0      0
Title           0
Video ID        0
Published At    0
Keyword         0
Likes           0
Comments        0
Views           0
dtype: int64


In [9]:
print(df.head())

   Unnamed: 0     Title     Video ID Published At  Keyword     Likes  \
0           0  0.138694  wAZZ-UWGVHI   2022-08-23    0.925  0.000207   
1           1  0.826228  b3x28s61q3c   2022-08-24    0.925  0.004669   
2           2  0.598489  4mgePWWCAmA   2022-08-23    0.925  0.003881   
3           3  0.652995  kXiYSI7H2b0   2022-08-23    0.925  0.004352   
4           4  0.170534  ErMwWXQxHp0   2022-08-08    0.925  0.005869   

   Comments     Views  
0  0.000918  0.000034  
1  0.005877  0.000436  
2  0.004556  0.000388  
3  0.001947  0.000229  
4  0.007036  0.000460  


In [10]:
X = df[['Likes', 'Comments', 'Title', 'Keyword']]  #Features
y = df['Views']  #Target

In [11]:
#Split dei dati in test e train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Linear Regression

In [12]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

#Random Forest Default

In [13]:
rf_model_default = RandomForestRegressor(random_state=42)
rf_model_default.fit(X_train, y_train)
y_pred_rf_default = rf_model_default.predict(X_test)

#Random Forest Avanzato


In [14]:
rf_model_advanced = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model_advanced.fit(X_train, y_train)
y_pred_rf_advanced = rf_model_advanced.predict(X_test)

In [16]:
#Grid Search per Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf_model_gs = RandomForestRegressor(random_state=42)
rf_grid_search = GridSearchCV(estimator=rf_model_gs, param_grid=rf_param_grid,
                              cv=3, n_jobs=-1, verbose=2, scoring='r2')
rf_grid_search.fit(X_train, y_train)

print("Miglior modello Random Forest:")
print(rf_grid_search.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


KeyboardInterrupt: 

#Decision Tree

In [17]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

#Gradient Boosting

In [18]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

In [19]:
#Grid Search per Gradient Boosting
gb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gb_model_gs = GradientBoostingRegressor(random_state=42)
gb_grid_search = GridSearchCV(estimator=gb_model_gs, param_grid=gb_param_grid,
                              cv=3, n_jobs=-1, verbose=2, scoring='r2')
gb_grid_search.fit(X_train, y_train)

print("Miglior modello Gradient Boosting:")
print(gb_grid_search.best_params_)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


KeyboardInterrupt: 

#Performance dei modelli


In [22]:
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  #Radice quadrata del MSE
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

#Valutazione dei modelli
print("Valutazione dei modelli:")
evaluate_model(y_test, y_pred_linear, "Regressione Lineare")
evaluate_model(y_test, y_pred_rf_default, "Random Forest (default)")
evaluate_model(y_test, y_pred_rf_advanced, "Random Forest (iperparametri avanzati senza grid search)")
evaluate_model(y_test, y_pred_dt, "Decision Tree")
evaluate_model(y_test, y_pred_gb, "Gradient Boosting senza grid search")

Valutazione dei modelli:
Regressione Lineare - MSE: 0.0000, RMSE: 0.0002, MAE: 0.0001, R²: 0.6569
Random Forest (default) - MSE: 0.0000, RMSE: 0.0002, MAE: 0.0001, R²: 0.5813
Random Forest (iperparametri avanzati senza grid search) - MSE: 0.0000, RMSE: 0.0002, MAE: 0.0001, R²: 0.5745
Decision Tree - MSE: 0.0000, RMSE: 0.0003, MAE: 0.0001, R²: 0.1196
Gradient Boosting senza grid search - MSE: 0.0000, RMSE: 0.0002, MAE: 0.0001, R²: 0.6627


#Implementazione del progetto in ambito reale

In [24]:
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive')

df = pd.read_csv('/content/drive/MyDrive/dataset/videos-stats.csv', encoding='latin-1')

print(f"Dimensioni del dataset: {df.shape}")
print(f"Totale righe: {df.shape[0]}")
print(f"Totale colonne: {df.shape[1]}")

print(df.head())

df = pd.read_csv('/content/drive/MyDrive/dataset/videos-stats.csv', encoding='latin-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dimensioni del dataset: (1881, 8)
Totale righe: 1881
Totale colonne: 8
   Unnamed: 0                                              Title     Video ID  \
0           0  Apple Pay Is Killing the Physical Wallet After...  wAZZ-UWGVHI   
1           1                    The most EXPENSIVE thing I own.  b3x28s61q3c   
2           2                 My New House Gaming Setup is SICK!  4mgePWWCAmA   
3           3  Petrol Vs Liquid Nitrogen | Freezing Experimen...  kXiYSI7H2b0   
4           4                     Best Back to School Tech 2022!  ErMwWXQxHp0   

  Published At Keyword    Likes  Comments      Views  
0   2022-08-23    tech   3407.0     672.0   135612.0  
1   2022-08-24    tech  76779.0    4306.0  1758063.0  
2   2022-08-23    tech  63825.0    3338.0  1564007.0  
3   2022-08-23    tech  71566.0    1426.0   922918.0  
4   2022-08-08    tech  96513.0    515

In [25]:
#Selezionate le colonne da normalizzare
columns_to_normalize = ['Likes','Comments','Views','Title','Keyword']

In [26]:
#Visualizzare i primi risultati
print(df.head())

   Unnamed: 0                                              Title     Video ID  \
0           0  Apple Pay Is Killing the Physical Wallet After...  wAZZ-UWGVHI   
1           1                    The most EXPENSIVE thing I own.  b3x28s61q3c   
2           2                 My New House Gaming Setup is SICK!  4mgePWWCAmA   
3           3  Petrol Vs Liquid Nitrogen | Freezing Experimen...  kXiYSI7H2b0   
4           4                     Best Back to School Tech 2022!  ErMwWXQxHp0   

  Published At Keyword    Likes  Comments      Views  
0   2022-08-23    tech   3407.0     672.0   135612.0  
1   2022-08-24    tech  76779.0    4306.0  1758063.0  
2   2022-08-23    tech  63825.0    3338.0  1564007.0  
3   2022-08-23    tech  71566.0    1426.0   922918.0  
4   2022-08-08    tech  96513.0    5155.0  1855644.0  


In [27]:
#Imputare valori mancanti
#Numerici: sostituire con la media per "Likes", "Comments" e "Views"
for col in ["Likes", "Comments", "Views"]:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

#Rimuovere duplicati
df = df.drop_duplicates()

#GESTIONE DEGLI OUTLIERS
#Identificazione di outlier con il metodo iqr
numeric_cols = df.select_dtypes(include=np.number).columns
if not df.empty:
    q1 = df[numeric_cols].quantile(0.25)
    q3 = df[numeric_cols].quantile(0.75)
    iqr = q3 - q1

    # Rimuovere outlier solo dalle colonne numeriche
    outliers = ((df[numeric_cols] < (q1 - 1.5 * iqr)) | (df[numeric_cols] > (q3 + 1.5 * iqr))).any(axis=1)
    df = df[~outliers]

#Controllo valori mancanti
missing_values = df.isnull().sum()
print("Valori mancanti per colonna:")
print(missing_values)

#Rimozione delle righe con valori mancanti
df.dropna(inplace=True)

Valori mancanti per colonna:
Unnamed: 0      0
Title           0
Video ID        0
Published At    0
Keyword         0
Likes           0
Comments        0
Views           0
dtype: int64


In [28]:
X = df[['Likes', 'Comments', 'Title', 'Keyword']]  #Features
y = df['Views']  #Target

#Suddivisione del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
#Creazione del preprocessor con OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Keyword'])
    ]
)

#Creazione del modello di Gradient Boosting
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor())
])

In [30]:
#Addestramento del modello
pipeline.fit(X_train, y_train)

#Stampa Keyword disponibili
print("le Keyword disponibili sono: ")
valori_unici = df['Keyword'].unique()
print(valori_unici)

#Funzione per fare previsioni
def predici_visualizzazioni(keyword):
    #Controlla se la keyword esiste nel dataset
    if keyword not in df['Keyword'].values:
        print(f"Attenzione: la keyword '{keyword}' non è presente nel dataset.")
        return None  #Restituisci None se la keyword non esiste

    keyword_df = pd.DataFrame({'Keyword': [keyword]})
    previsione = pipeline.predict(keyword_df)
    return previsione[0]

le Keyword disponibili sono: 
['tech' 'news' 'gaming' 'sports' 'how-to' 'business' 'tutorial' 'finance'
 'crypto' 'cubes' 'chess' 'apple' 'google' 'interview' 'history'
 'literature' 'bed' 'trolling' 'asmr' 'music' 'food' 'mukbang' 'reaction'
 'lofi' 'animals' 'education' 'minecraft' 'sat' 'mrbeast' 'cnn'
 'mathchemistry' 'biology' 'physics' 'computer science' 'nintendo' 'xbox'
 'marvel' 'movies' 'data science' 'game development' 'machine learning']


In [ ]:
#Richiesta all'utente
while True:
    keyword_input = input("Inserisci il nome di una Keyword (o 'exit' per uscire): ")
    if keyword_input.lower() == 'exit':
        break
    visualizzazioni_predette = predici_visualizzazioni(keyword_input)

    if visualizzazioni_predette is not None:  #Solo se la previsione è valida
        print(f"Le visualizzazioni previste per la categoria '{keyword_input}' sono: {visualizzazioni_predette:.2f}")

Le visualizzazioni previste per la categoria 'minecraft' sono: 790417.12
